In [67]:
import math
from web3 import Web3, AsyncWeb3
from web3 import eth
import abi as abi_import
import time
from datetime import datetime

In [3]:
w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/HsRAh-rTKhWRIzJAVCfUU3VXFqilsfLf'))
w3.is_connected()

True

In [4]:
sy_addr = '0xED97f94dd94255637A054098604E0201C442a3FD'
sy_abi = abi_import.sy_token_abi

mkt_abi = abi_import.mkt
mkt_addr = '0xC9beCdbC62efb867cB52222b34c187fB170379C6'


In [5]:
contract_instance = w3.eth.contract(address=mkt_addr, abi=mkt_abi)

In [6]:
mkt_data = contract_instance.functions._storage().call()

In [7]:
total_pt = mkt_data[0]
total_sy = mkt_data[1]
impliedRate = mkt_data[2]

In [8]:
pt_eth =  float(Web3.from_wei(total_pt, 'ether'))
sy_eth =  float(Web3.from_wei(total_sy, 'ether'))

In [9]:
pt_ratio = pt_eth / (pt_eth + sy_eth)
px_ratio = pt_ratio / (1-pt_ratio)

In [10]:
ln_lastImpliedRate = float(Web3.from_wei(impliedRate, 'ether'))
lastImpliedRate = math.exp(ln_lastImpliedRate)

In [11]:
scalar_root =  contract_instance.functions.scalarRoot().call()

In [12]:
initAnchor = contract_instance.functions.initialAnchor().call()

In [13]:
initAnchor_eth = float(Web3.from_wei(initAnchor, 'ether'))
scalar_root_eth = float(Web3.from_wei(scalar_root, 'ether'))

In [14]:
startTime = datetime(2023, 10, 11, 3, 19, 11).timestamp()
startTime

1697008751.0

In [15]:
expiry = contract_instance.functions.expiry().call()
unixYear = 31556926

In [16]:
(expiry - startTime) / unixYear

0.4618716347720307

In [17]:
def update_now():
    return time.time()

def rateScalar(nowTime):
    return scalar_root_eth / (nowTime / unixYear)

def yrsLeft(nowTime):
    return (expiry - nowTime) / unixYear

def rateAnchor(nowTime):
    return lastImpliedRate**yrsLeft(nowTime) - (math.log(pt_ratio/(1-pt_ratio))) / rateScalar(nowTime)

In [18]:
rtScalar = rateScalar(update_now())
rtAnchor = rateAnchor(update_now())

In [19]:
exchange_rt_px = ((math.log(pt_ratio/(1-pt_ratio))) / rtScalar) + rtAnchor

In [20]:
pt_price = 1 / exchange_rt_px
sy_price = pt_price * ((lastImpliedRate)**(yrsLeft(update_now())) - 1)

In [21]:
silo =  '[{"inputs":[{"internalType":"contract ISiloRepository","name":"_siloRepo","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"DifferentArrayLength","type":"error"},{"inputs":[],"name":"InvalidRepository","type":"error"},{"inputs":[],"name":"UnsupportedLTVType","type":"error"},{"inputs":[],"name":"UserIsZero","type":"error"},{"inputs":[],"name":"ZeroAssets","type":"error"},{"inputs":[{"internalType":"uint256","name":"_assetTotalDeposits","type":"uint256"},{"internalType":"contract IShareToken","name":"_shareToken","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"balanceOfUnderlying","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"borrowAPY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"borrowShare","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"calcFee","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"calculateBorrowValue","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"calculateCollateralValue","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"collateralBalanceOfUnderlying","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"collateralOnlyDeposits","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"debtBalanceOfUnderlying","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"depositAPY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"uint256","name":"_timestamp","type":"uint256"}],"name":"getBorrowAmount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"address","name":"_user","type":"address"},{"internalType":"uint256","name":"_timestamp","type":"uint256"}],"name":"getDepositAmount","outputs":[{"internalType":"uint256","name":"totalUserDeposits","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"getModel","outputs":[{"internalType":"contract IInterestRateModel","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"getUserLTV","outputs":[{"internalType":"uint256","name":"userLTV","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"getUserLiquidationThreshold","outputs":[{"internalType":"uint256","name":"liquidationThreshold","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"getUserMaximumLTV","outputs":[{"internalType":"uint256","name":"maximumLTV","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"getUtilization","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"hasPosition","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"inDebt","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_user","type":"address"}],"name":"isSolvent","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"lensPing","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"liquidity","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"protocolFees","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"siloRepository","outputs":[{"internalType":"contract ISiloRepository","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalBorrowAmount","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalBorrowAmountWithInterest","outputs":[{"internalType":"uint256","name":"_totalBorrowAmount","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalBorrowShare","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalDeposits","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"contract ISilo","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"totalDepositsWithInterest","outputs":[{"internalType":"uint256","name":"_totalDeposits","type":"uint256"}],"stateMutability":"view","type":"function"}]'


silo_addr = '0x32a4Bcd8DEa5E18a12a50584682f8E4B77fFF2DF'

siloYield = '0x96eFdF95Cc47fe90e8f63D2f5Ef9FB8B180dAeB9'
yieldToken = '0xD533a949740bb3306d119CC777fa900bA034cd52' # crv
silo_contract = w3.eth.contract(address=silo_addr, abi=silo)


In [22]:
borrowAPY = silo_contract.functions.borrowAPY(siloYield, yieldToken).call()

In [23]:
borrowAPY_eth = float(Web3.from_wei(borrowAPY, 'ether'))
borrowAPY_eth

0.069180134581872

In [24]:
depositAPY = silo_contract.functions.depositAPY(siloYield, yieldToken).call()
depositAPY_eth = float(Web3.from_wei(depositAPY, 'ether'))
depositAPY_eth

0.03418565121037136

In [25]:
util = silo_contract.functions.getUtilization(siloYield, yieldToken).call()
util_eth = float(Web3.from_wei(util,'ether'))
util_eth

0.49414381843903366

In [26]:
liq = silo_contract.functions.liquidity(siloYield, yieldToken).call()
liq_eth = float(Web3.from_wei(liq,'ether'))
liq_eth

4236322.310791544

In [27]:
borrow = silo_contract.functions.totalBorrowAmount(siloYield, yieldToken).call()
borrow_eth = float(Web3.from_wei(borrow,'ether'))
borrow_eth

4132055.192710716

In [28]:
deposits = silo_contract.functions.totalDeposits(siloYield, yieldToken).call()
deposits_eth = float(Web3.from_wei(deposits,'ether'))
deposits_eth

8362049.748519761

In [37]:
crvUSD = '0xf939E0A03FB07F59A73314E73794Be0E57ac1b4E'

borrowAPY_crvUSD = float(Web3.from_wei(silo_contract.functions.borrowAPY(siloYield, crvUSD).call(), 'ether'))
depositAPY_crvUSD = float(Web3.from_wei(silo_contract.functions.depositAPY(siloYield, crvUSD).call(), 'ether'))
util_crvUSD = float(Web3.from_wei(silo_contract.functions.getUtilization(siloYield, crvUSD).call(), 'ether'))

liq_crvUSD= float(Web3.from_wei(silo_contract.functions.liquidity(siloYield, crvUSD).call(), 'ether'))

borrow_crvUSD = float(Web3.from_wei(silo_contract.functions.totalBorrowAmount(siloYield, crvUSD).call(), 'ether'))

deposits_crvUSD = float(Web3.from_wei(silo_contract.functions.totalDeposits(siloYield, crvUSD).call(), 'ether'))

In [38]:
print("borrow apy: ", borrowAPY_crvUSD)
print("deposit apy: ", depositAPY_crvUSD)
print("util: ", util_crvUSD)
print("liq: ", liq_crvUSD)
print("borrows: ", borrow_crvUSD)
print("deposits: ", deposits_crvUSD)

borrow apy:  0.068699768138352
deposit apy:  0.04719664005420262
util:  0.6869976816696566
liq:  12764449.286557846
borrows:  27921253.49027689
deposits:  40642427.52962135


In [54]:
interestRateModel = silo_contract.functions.getModel(siloYield, crvUSD).call()

In [56]:
model_abi = '[{"inputs":[{"components":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"internalType":"struct IInterestRateModel.Config","name":"_config","type":"tuple"},{"internalType":"address","name":"_owner","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"InvalidBeta","type":"error"},{"inputs":[],"name":"InvalidKcrit","type":"error"},{"inputs":[],"name":"InvalidKi","type":"error"},{"inputs":[],"name":"InvalidKlin","type":"error"},{"inputs":[],"name":"InvalidKlow","type":"error"},{"inputs":[],"name":"InvalidRi","type":"error"},{"inputs":[],"name":"InvalidTcrit","type":"error"},{"inputs":[],"name":"InvalidTimestamps","type":"error"},{"inputs":[],"name":"InvalidUcrit","type":"error"},{"inputs":[],"name":"InvalidUlow","type":"error"},{"inputs":[],"name":"InvalidUopt","type":"error"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"silo","type":"address"},{"indexed":true,"internalType":"address","name":"asset","type":"address"},{"components":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"indexed":false,"internalType":"struct IInterestRateModel.Config","name":"config","type":"tuple"}],"name":"ConfigUpdate","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newPendingOwner","type":"address"}],"name":"OwnershipPending","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"inputs":[],"name":"ASSET_DATA_OVERFLOW_LIMIT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"DP","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RCOMP_MAX","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"X_MAX","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"components":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"internalType":"struct IInterestRateModel.Config","name":"_c","type":"tuple"},{"internalType":"uint256","name":"_totalDeposits","type":"uint256"},{"internalType":"uint256","name":"_totalBorrowAmount","type":"uint256"},{"internalType":"uint256","name":"_interestRateTimestamp","type":"uint256"},{"internalType":"uint256","name":"_blockTimestamp","type":"uint256"}],"name":"calculateCompoundInterestRate","outputs":[{"internalType":"uint256","name":"rcomp","type":"uint256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"components":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"internalType":"struct IInterestRateModel.Config","name":"_c","type":"tuple"},{"internalType":"uint256","name":"_totalDeposits","type":"uint256"},{"internalType":"uint256","name":"_totalBorrowAmount","type":"uint256"},{"internalType":"uint256","name":"_interestRateTimestamp","type":"uint256"},{"internalType":"uint256","name":"_blockTimestamp","type":"uint256"}],"name":"calculateCompoundInterestRateWithOverflowDetection","outputs":[{"internalType":"uint256","name":"rcomp","type":"uint256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"},{"internalType":"bool","name":"overflow","type":"bool"}],"stateMutability":"pure","type":"function"},{"inputs":[{"components":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"internalType":"struct IInterestRateModel.Config","name":"_c","type":"tuple"},{"internalType":"uint256","name":"_totalDeposits","type":"uint256"},{"internalType":"uint256","name":"_totalBorrowAmount","type":"uint256"},{"internalType":"uint256","name":"_interestRateTimestamp","type":"uint256"},{"internalType":"uint256","name":"_blockTimestamp","type":"uint256"}],"name":"calculateCurrentInterestRate","outputs":[{"internalType":"uint256","name":"rcur","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"config","outputs":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"uint256","name":"_blockTimestamp","type":"uint256"}],"name":"getCompoundInterestRate","outputs":[{"internalType":"uint256","name":"rcomp","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_asset","type":"address"},{"internalType":"uint256","name":"_blockTimestamp","type":"uint256"}],"name":"getCompoundInterestRateAndUpdate","outputs":[{"internalType":"uint256","name":"rcomp","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"}],"name":"getConfig","outputs":[{"components":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"internalType":"struct IInterestRateModel.Config","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"uint256","name":"_blockTimestamp","type":"uint256"}],"name":"getCurrentInterestRate","outputs":[{"internalType":"uint256","name":"rcur","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"interestRateModelPing","outputs":[{"internalType":"bytes4","name":"","type":"bytes4"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address[]","name":"_silos","type":"address[]"},{"internalType":"contract ISiloRepository","name":"_siloRepository","type":"address"}],"name":"migrationFromV1","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"internalType":"uint256","name":"_blockTimestamp","type":"uint256"}],"name":"overflowDetected","outputs":[{"internalType":"bool","name":"overflow","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"pendingOwner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"removePendingOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_silo","type":"address"},{"internalType":"address","name":"_asset","type":"address"},{"components":[{"internalType":"int256","name":"uopt","type":"int256"},{"internalType":"int256","name":"ucrit","type":"int256"},{"internalType":"int256","name":"ulow","type":"int256"},{"internalType":"int256","name":"ki","type":"int256"},{"internalType":"int256","name":"kcrit","type":"int256"},{"internalType":"int256","name":"klow","type":"int256"},{"internalType":"int256","name":"klin","type":"int256"},{"internalType":"int256","name":"beta","type":"int256"},{"internalType":"int256","name":"ri","type":"int256"},{"internalType":"int256","name":"Tcrit","type":"int256"}],"internalType":"struct IInterestRateModel.Config","name":"_config","type":"tuple"}],"name":"setConfig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newPendingOwner","type":"address"}],"name":"transferPendingOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"}]'
model_contract = w3.eth.contract(address=interestRateModel, abi=model_abi)

In [58]:
dp = model_contract.functions.DP().call()
rcomp_max = model_contract.functions.RCOMP_MAX().call()
x_max = model_contract.functions.X_MAX().call()


In [59]:
config = model_contract.functions.getConfig(siloYield, crvUSD).call()

In [60]:
config # upot, ucrit, ulow, ki, kcrit, klow, klin, beta, ri, Tcrit

(800000000000000000,
 900000000000000000,
 600000000000000000,
 183506,
 237823439878,
 26424826653,
 3170979198,
 27777777777778,
 2178464807,
 0)

In [61]:
uopt = config[0]
ucrit = config[1]
ulow = config[2]
ki = config[3]
kcrit = config[4]
klow = config[5]
klin = config[6]
beta = config[7]
ri = config[8]
Tcrit = config[9]

In [62]:
cb = beta * 3600
clin = klin * 365 * 24 * 3600 * uopt
clow = klow * 365 * 24 * 3600 * ulow
ccrit = 365 * 24 * 3600 * (1.0 - ulow)
ci = ki * 365 * (24**2) * (3600**2) * (1.0 - uopt)

In [63]:
def latestBlockTS():
    return w3.eth.get_block('latest')['timestamp']

def currInterestRate(t0, t1, _ri, u, Tcrit):


1697543987